<a href="https://colab.research.google.com/github/MixalisBaltzakis/commit_for_third_exercise/blob/main/parallila3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

python script για μετατροπη του .mat. Δημιουργουμε ενα φακελο με ονομα το ονομα του γραφου που περιεχει το row_ptr.bin col_ind.bin meta.txt

In [ ]:
import sys
import os
import numpy as np
import scipy.io
import h5py

def convert(mat_file):
    # 1. Βρες το όνομα του φακέλου (χωρίς το .mat)
    base_name = os.path.splitext(os.path.basename(mat_file))[0]

    # 2. Φτιάξε τον φάκελο (αν δεν υπάρχει)
    print(f"Creating directory: {base_name}/")
    os.makedirs(base_name, exist_ok=True)

    print(f"Checking file format for: {mat_file}...")

    # --- (Η λογική φόρτωσης μένει ίδια - Scipy vs H5PY) ---
    try:
        mat = scipy.io.loadmat(mat_file)
        print("Detected Version: < v7.3 (Standard)")
        if 'Problem' in mat:
            A = mat['Problem']['A'][0][0]
        else:
            keys = [k for k in mat.keys() if not k.startswith('_')]
            A = mat[keys[0]]
        indptr = A.indptr
        indices = A.indices
        n = A.shape[0]
        nnz = A.nnz
    except NotImplementedError:
        print("Detected Version: v7.3 (HDF5 Based)")
        with h5py.File(mat_file, 'r') as f:
            if 'Problem' in f:
                group = f['Problem']['A']
            else:
                key = list(f.keys())[0]
                group = f[key]
            indptr = np.array(group['jc']).flatten()
            indices = np.array(group['ir']).flatten()
            n = len(indptr) - 1
            nnz = len(indices)

    print(f"Graph loaded! Nodes: {n}, Edges: {nnz}")

    # --- ΑΠΟΘΗΚΕΥΣΗ ΣΤΟΝ ΦΑΚΕΛΟ ---

    # Συνθέτουμε τα paths: π.χ. "file1/row_ptr.bin"
    path_row = os.path.join(base_name, "row_ptr.bin")
    path_col = os.path.join(base_name, "col_ind.bin")
    path_meta = os.path.join(base_name, "meta.txt")

    print(f"Saving to {path_row} (int64)...")
    indptr.astype(np.int64).tofile(path_row)

    print(f"Saving to {path_col} (int32)...")
    indices.astype(np.int32).tofile(path_col)

    print(f"Saving to {path_meta}...")
    with open(path_meta, "w") as f:
        f.write(f"{n}\n{nnz}")

    print(f"Done! All files are inside the folder: {base_name}/")



Απλο wget και μετατροπη αν θελουμε να διαγραφουν τα αρχεια οταν τελειωσει το runtime

In [ ]:
!wget https://suitesparse-collection-website.herokuapp.com/mat/GenBank/kmer_A2a.mat
!wget https://suitesparse-collection-website.herokuapp.com/mat/DIMACS10/rgg_n_2_24_s0.mat
!wget https://suitesparse-collection-website.herokuapp.com/mat/Janna/Queen_4147.mat

convert("kmer_A2a.mat")
convert("rgg_n_2_24_s0.mat")
convert("Queen_4147.mat")

--2026-01-16 22:11:41--  https://suitesparse-collection-website.herokuapp.com/mat/GenBank/kmer_A2a.mat
Resolving suitesparse-collection-website.herokuapp.com (suitesparse-collection-website.herokuapp.com)... 34.201.81.34, 54.208.186.182, 54.224.34.30, ...
Connecting to suitesparse-collection-website.herokuapp.com (suitesparse-collection-website.herokuapp.com)|34.201.81.34|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://sparse-files.engr.tamu.edu/mat/GenBank/kmer_A2a.mat [following]
--2026-01-16 22:11:42--  http://sparse-files.engr.tamu.edu/mat/GenBank/kmer_A2a.mat
Resolving sparse-files.engr.tamu.edu (sparse-files.engr.tamu.edu)... 3.5.129.168, 3.5.130.59, 52.219.102.240, ...
Connecting to sparse-files.engr.tamu.edu (sparse-files.engr.tamu.edu)|3.5.129.168|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1111577368 (1.0G) [binary/octet-stream]
Saving to: ‘kmer_A2a.mat’

kmer_A2a.mat        100%[==================

Ανεβαζουμε τον φακελο με τα binary αρχεια και το .txt στο drive ωστε να μη χρειαζεται να κανουμε ξανα μετατροπη οταν τελειωσει το runtime. Φορτωνουμε απο το drive τα αρχεια τοπικα για γρηγοροτερη αναγνωση

Κανω comment out τον κωδικα

In [7]:
#import os
#import shutil
#%cd /content/drive/MyDrive/
#links = [
#  "https://suitesparse-collection-website.herokuapp.com/mat/GenBank/kmer_A2a.mat",
#  "https://suitesparse-collection-website.herokuapp.com/mat/DIMACS10/rgg_n_2_24_s0.mat",
#  "https://suitesparse-collection-website.herokuapp.com/mat/Janna/Queen_4147.mat"
#]
#
#base_names = [os.path.basename(link) for link in links]
#names = [base_name.removesuffix('.mat') for base_name in base_names]
#mat_destination_paths = [('/content/drive/MyDrive/'+ base_name) for base_name in base_names]
#destination_paths = [('/content/drive/MyDrive/'+ name) for name in names]
#
#for destination_path, link, base_name in zip(mat_destination_paths, links, base_names):
#  #Βλεπουμε αν υπρχει το .mat στον drive
#  if not os.path.exists(destination_path):
#      print(f"Downolading {base_name}")
#      !wget "{link}"
#      print("Done!")
#  else:
#    print(".mat already exists on drive no need to download.")
#
#for destination_path, base_name in zip(destination_paths, base_names):
#  #Βλεπουμε αν υπρχει ο φακελος με τα binaries στο drive
#  if not os.path.exists(destination_path):
#      print(f"Converting {base_name} to binary")
#      convert(base_name)
#      print("Done!")
#  else:
#    print(".mat file already converted to binary")
#
#mat_source_paths = [('/content/drive/MyDrive/'+ base_name) for base_name in base_names]
#source_paths = [('/content/drive/MyDrive/'+ name) for name in names]
#mat_destination_paths = [('/content/'+ base_name) for base_name in base_names]
#destination_paths = [('/content/'+ name) for name in names]
#
## Ελέγχουμε αν υπάρχει ήδη το .mat(για να μην το ξανακάνουμε αν κάνουμε restart το runtime)
##for destination_path, source_path in zip(mat_destination_paths, mat_source_paths):
##  if not os.path.exists(destination_path):
##      print("Copying file from Drive to local VM...")
##      shutil.copy(source_path, destination_path)
##      print("Done!")
##  else:
##    print(".mat file already exists locally no need to copy.")
#
##Το ιδιο και για τον φακελο απο το drive
#for destination_path, source_path in zip(destination_paths, source_paths):
#  if not os.path.exists(destination_path):
#      print("Copying folder from Drive to local VM...")
#      shutil.copytree(source_path, destination_path)
#      print("Done!")
#  else:
#    print("Folder already exists locally no need to copy.")
#
#
#
#%cd /content/

Κωδικας


In [ ]:
%%writefile GPU_cc.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <time.h>
#include <string.h>
#include <limits.h>

#define WARP_SIZE 32

// --- HEADER ΓΙΑ ERROR CHECKING
#define checkCuda(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true) {
   if (code != cudaSuccess) {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}
// ----------------------------------------------------

typedef struct {
    int n;
    long long nnz;
    long long *row_ptr;
    int *col_ind;
} CSCMatrix;

CSCMatrix *load_problem_serial(const char *dataset_name) {
    char filepath[512];
    int n;
    long long nnz;

    // 1. Διάβασμα Metadata
    sprintf(filepath, "%s/meta.txt", dataset_name);
    FILE *f = fopen(filepath, "r");
    if (!f) { printf("Error opening meta.txt\n"); exit(1); }
    fscanf(f, "%d\n%lld", &n, &nnz);
    fclose(f);

    printf("Loading Graph: n=%d, nnz=%lld\n", n, nnz);

    // Δέσμευση του struct
    CSCMatrix *A = (CSCMatrix *)malloc(sizeof(CSCMatrix));
    A->n = n;
    A->nnz = nnz;

    // Διάβασμα row_ptr.bin
    A->row_ptr = (long long *)malloc((n + 1) * sizeof(long long));
    sprintf(filepath, "%s/row_ptr.bin", dataset_name);

    f = fopen(filepath, "rb"); // "rb" για binary read
    if (!f) { printf("Error opening row_ptr.bin\n"); exit(1); }

    // Διαβάζουμε n+1 στοιχεία
    size_t read_count = fread(A->row_ptr, sizeof(long long), n + 1, f);
    if (read_count != n + 1) { printf("Error reading row_ptr!\n"); exit(1); }
    fclose(f);

    // 3. Διάβασμα col_ind.bin
    A->col_ind = (int *)malloc(nnz * sizeof(int));
    sprintf(filepath, "%s/col_ind.bin", dataset_name);

    f = fopen(filepath, "rb");
    if (!f) { printf("Error opening col_ind.bin\n"); exit(1); }

    // Διαβάζουμε nnz στοιχεία
    read_count = fread(A->col_ind, sizeof(int), nnz, f);
    if (read_count != nnz) { printf("Error reading col_ind!\n"); exit(1); }
    fclose(f);

    return A;
}

void free_csc(CSCMatrix *A) {
    free(A->row_ptr);
    free(A->col_ind);
    free(A);
}

int vector_min(const int *v, int n, int min_val) {
    for (int i = 0; i < n; i++) {
        if (v[i] < min_val)
            min_val = v[i];
    }
    return min_val;
}

void print_vector(int *v, int n){
    printf("[ ");
    for (int i = 0; i < n; i++) {
        printf("%d ", v[i]);
    }
    printf("]\n");
}

void radixSort(int *arr, size_t n) {
    if (n == 0) return;

    // 1. Δέσμευση μνήμης ΜΙΑ φορά για τον buffer
    int *buffer = (int *)malloc(n * sizeof(int));
    if (!buffer) {
        printf("Malloc failed\n");
        return;
    }

    // pointers για ping-pong (εναλλαγή πηγής-προορισμού)
    int *src = arr;
    int *dest = buffer;

    // Τρέχουμε ακριβώς 4 φορές (γιατί sizeof(int) = 4 bytes = 32 bits)
    // i = 0 (shift 0), i = 1 (shift 8), i = 2 (shift 16), i = 3 (shift 24)
    for (int i = 0; i < 4; i++) {
        unsigned int count[256] = {0};
        int shift = i * 8;

        // Βήμα 1: Histogram (Μέτρημα) - Χωρίς διαιρέσεις, μόνο Bitwise
        for (size_t j = 0; j < n; j++) {
            // Παίρνουμε το συγκεκριμένο byte (0-255)
            unsigned char byte = (src[j] >> shift) & 0xFF;
            count[byte]++;
        }

        // Βήμα 2: Prefix Sum (Υπολογισμός θέσεων)
        unsigned int total = 0;
        for (int k = 0; k < 256; k++) {
            unsigned int old_count = count[k];
            count[k] = total;
            total += old_count;
        }

        // Βήμα 3: Τοποθέτηση στον πίνακα προορισμού
        for (size_t j = 0; j < n; j++) {
            unsigned char byte = (src[j] >> shift) & 0xFF;
            dest[count[byte]++] = src[j];
        }

        // Swap pointers: Ο προορισμός γίνεται πηγή για τον επόμενο γύρο
        int *temp = src;
        src = dest;
        dest = temp;
    }

    // Επειδή κάναμε 4 περάσματα (ζυγός αριθμός), τα δεδομένα κατέληξαν στο 'src'.
    // Αν το 'src' τυχαίνει να είναι ο buffer (που δεν θα είναι με 4 περάσματα, αλλά για σιγουριά),
    // αντιγράφουμε πίσω.
    // ΣΗΜΕΙΩΣΗ: Με 4 loops:
    // Start: src=arr, dest=buf
    // End Loop 1: src=buf, dest=arr
    // End Loop 2: src=arr, dest=buf
    // End Loop 3: src=buf, dest=arr
    // End Loop 4: src=arr, dest=buf  <-- ΤΑ ΔΕΔΟΜΕΝΑ ΕΙΝΑΙ ΣΤΟΝ ARR

    // Απλά ελευθερώνουμε τον buffer.
    free(buffer);
}

size_t sort_unique_int(int *v, size_t n) {
    if (n == 0) return 0;


    radixSort(v, n);

    // Unique Logic
    size_t w = 1;
    for (size_t r = 1; r < n; ++r) {
        if (v[r] != v[w - 1]) {
            v[w] = v[r];
            ++w;
        }
    }
    return w;
}

double wall_time(void) {
    struct timespec ts;
    clock_gettime(CLOCK_MONOTONIC, &ts);
    return (double)ts.tv_sec + (double)ts.tv_nsec * 1e-9;
}

__global__ void warp_per_row_kernel(int* d_labels, const long long* row_ptr, const int *col_ind, int node_number, bool *d_changed) {
    // Βρίσκουμε ποιο thread είμαστε Global
    long long tid = (long long)threadIdx.x + (long long)blockIdx.x * (long long)blockDim.x;
    long long global_warp_id = tid / WARP_SIZE;
    int warp_id = (int)global_warp_id; // Ποια γραμμή αναλαμβάνουμε
    int lane = tid % WARP_SIZE;    // Ποιο thread είμαστε μέσα στο Warp (0-31)

    if (warp_id >= node_number) return;

    long long start = row_ptr[warp_id];
    long long end = row_ptr[warp_id + 1];

    if (start == end)   return;
    if (d_labels[warp_id] == 0 ) return;

    int my_min = d_labels[warp_id];

    // Το καθε thread διαβαζει ενα γειτονα και επειτα διαβαζει αυτον που ειναι 32 θεσεις μακρια
    for (long long i = start + lane; i < end; i += WARP_SIZE) {
        int neighbor = col_ind[i];
        int neighbor_label = d_labels[neighbor];

        // Κρατάμε το μικρότερο που είδαμε
        if (neighbor_label < my_min) {
            my_min = neighbor_label;
        }
    }

    // Warp Reduction (Βρίσκουμε το ελάχιστο μεταξύ των 32 threads).Αυτό δουλεύει σε επίπεδο καταχωρητών. Πρωτο βημα παιρνουμε την ελαχιστη τιμη της συγκρισης 0-16, 1-17, 2-18... Στο δευτερο παιρνουμε το ελαχιστο του 0-8, 1-9, 2-10 και παει λεγοντας. Στο τελος το thrread 0 εχει το ελαχιστο
    unsigned mask = __activemask();
    for (int offset = WARP_SIZE/2 ; offset > 0 ; offset /= 2) {
        int other_val = __shfl_down_sync(mask, my_min, offset);
        if (other_val < my_min) {
            my_min = other_val;
        }
    }


    // Μόνο το πρώτο thread του Warp γράφει, γιατι εκεινο εχει την ελαχιστη τιμη, αν βρέθηκε κάτι μικρότερο και αλλαζει τη κατασταση του changed σε true
    if (lane == 0) {
      if (my_min < d_labels[warp_id]) {
        d_labels[warp_id] = my_min;
        *d_changed = true;
      }
    }
}

__global__ void thread_per_row_Kernel(int* d_labels, const long long* d_row_ptr, const int *d_col_ind, int node_number, bool *d_changed) {
    // Υπολογισμός του Global ID για κάθε thread
    int idx = blockDim.x * blockIdx.x + threadIdx.x;

    if (idx < node_number) {
      long long start = d_row_ptr[idx];
      long long end   = d_row_ptr[idx+1];
      int workers_node = idx;

      if (start == end)   return;
      if (d_labels[workers_node] == 0 ) return;

      int current_val = d_labels[workers_node];
      int min_val = d_labels[workers_node];
      for (long long i = start; i < end; ++i) {

        int nei = d_col_ind[i];
        int nei_label;

        nei_label = d_labels[nei];

        if (nei_label == 0) {
          min_val = 0;
          break;
          }
        if (nei_label < min_val) {
            min_val = nei_label;
            }
          }

        if (min_val < current_val) {
            d_labels[workers_node] = min_val;
            *d_changed = true; // Αλλαξε το labels αρα δε τελειωσαμε
        }
    }
}

__global__ void block_per_row_kernel(int* d_labels, const long long* d_row_ptr, const int *d_col_ind, int node_number, bool *d_changed) {
    //
    // Δηλωνουμε extern και χωρίς μέγεθος Η GPU θα περιμένει να της πουμε πόση μνήμη είναι αυτή κατά την κλήση του kernel
    extern __shared__ int s_warp_mins[];

    int row = blockIdx.x;
    if (row >= node_number) return;

    //Εδω το tid ειναι τοπικο στο block μας που ειναι υπευθυνο για καποιο συγκεκριμενο κομβο
    int tid = threadIdx.x;
    int warp_id = tid / WARP_SIZE; // σε ποιο warp του block ειμαστε
    int lane = tid % WARP_SIZE; // σε ποια θεση του warp ειμαστε

    // Υπολογιζουμε ποσα warps εχουμε σε ενα μπλοκ
    int num_warps = blockDim.x / WARP_SIZE;

    // Κανουμε κυκλικο περασμα. Οταν ενα block υπολογισει τη τιμη ενος node πηγαινει να υπολογισει τη τιμη του επομενου node που ειναι υπευθυνος
    // Αυτο το κανουμε γιατι αν ξεκινουσαμε blocks_number == node_numbers σε περιπτωση πολυ μεγαλου γραφου μας καθυστερει παρα πολυ
    // Τωρα διατηρουμε την ιδεα του block per row απλως τα ιδια blocks ειναι υπευθυνα για πολλους κομβους και οχι μονο για ενα

    for (int row = blockIdx.x; row < node_number; row += gridDim.x) {
    // Βρισκουμε το τοπικο ελαχιστο του καθε warp
    long long start = d_row_ptr[row];
    long long end = d_row_ptr[row + 1];
    int my_min = d_labels[row];

    if (start == end)  continue;
    if (d_labels[row] == 0 ) continue;

     // Το καθε thread διαβαζει ενα γειτονα και επειτα διαβαζει αυτον που ειναι block dim θεσεις μακρια
  for (long long i = start + tid ; i < end ; i += blockDim.x) {
        int neighbor = d_col_ind[i];
        int neighbor_label = d_labels[neighbor];

        // Κρατάμε το μικρότερο που είδαμε
        if (neighbor_label < my_min) {
            my_min = neighbor_label;
        }
    }

    // Warp Reduction
    for (int offset = 16; offset > 0; offset /= 2) {
        int other = __shfl_down_sync(0xFFFFFFFF, my_min, offset);
        if (other < my_min) my_min = other;
    }

    // Εγγραφή στη Dynamic Shared Memory. Καθε warp γραφει το τοπικο του ελαχιστο στο πινακα της shared memory.
    if (lane == 0) {
        s_warp_mins[warp_id] = my_min;
    }

    // Συγχρονισμος ωστε να εχει γραφτει η τιμη του αρχηγου του καθε warp στη shared memory
    __syncthreads();

    // Final Reduction το κανει το πρωτο warp του block
    if (warp_id == 0) {

      // Παιρνουμε το ελαχιστο του καθε warp. Το thread 0 εως το thread (num_warps-1) κραταει ενα τοπικο ελαχιστο το καθενα.
      // Τα υπολοιπα threads παιρνουν μια τεραστια ψευτικη τιμη στο block min ωστε να μην επηρερασουν στη συγκριση μετα.
      int block_min = (tid < num_warps) ? s_warp_mins[tid] : INT_MAX;

      // Παιρνω την ελαχιστη τιμη απο τις υποψηφιες που ειχα. Δε με ενοχλουν οι συγκρισεις με τα threads που δεν εχουν καποια πραγματικη τιμη στο block min καθως εχει ανατεθει το INT_MAX και θα αγνοηθουν
      unsigned mask = __activemask();
      for (int offset = 16; offset > 0; offset /= 2) {
          int other = __shfl_down_sync(mask, block_min, offset);
          if (other < block_min) block_min = other;
      }

      // Το thread 0 γραφει το ελαχιστο στο labels και τροποποιει τη κατασταση εξοδου καταλληλως.
      if (tid == 0) {
        if (block_min < d_labels[row]) {
          d_labels[row] = block_min;
          *d_changed = true;
        }
      }
    }
    // Συγχρονισμος για να μπουμε στην επομενη επαναληψη
    __syncthreads();
  }
}

int main(int argc, char *argv[]) {
  // Δινω το  ονομα του γραφου χωρις .mat σαν ορισμα
  char* filename = argv[1];
  char path[] = "/content";
  char full_path[200];

  snprintf(full_path, sizeof(full_path), "%s/%s", path, filename);
  printf("----------- %s -----------\n\n", argv[1]);
  // Δέσμευση και φόρτωση του γράφου στον Host
  CSCMatrix *h_graph = load_problem_serial(full_path);

  // Εργαλεια χρονομέτρησης αλγορίθμων
      cudaEvent_t *timer_start = (cudaEvent_t*)malloc(3*sizeof(cudaEvent_t));
      cudaEvent_t *timer_stop = (cudaEvent_t*)malloc(3*sizeof(cudaEvent_t));
      for(int i = 0; i<3 ; i++) {
        checkCuda(cudaEventCreate(&timer_start[i]));
        checkCuda(cudaEventCreate(&timer_stop[i]));
      }

  // Χαρακτηριστικα GPU
  int deviceId;
  cudaGetDevice(&deviceId); // Βρες σε ποια κάρτα τρέχουμε
  struct cudaDeviceProp props;
  cudaGetDeviceProperties(&props, deviceId); // Φέρε τα χαρακτηριστικά της
  int num_SMs = props.multiProcessorCount;  // Βρες πόσους SMs έχει η κάρτα
  printf("We have %d SMs in our GPU.\n\n", num_SMs);

  for(int strategy=0; strategy<3; strategy++){
      const int node_number = h_graph->n;
      long long nnz = h_graph->nnz;

      // Δέσμευση μνήμης στον Host
      bool h_changed;
      int *h_labels = (int*) malloc(node_number * sizeof(int));

      // Αρχικοποίηση δεδομένων
      for(int i =0; i<node_number; i++ ) {
          h_labels[i] = i;
      }

      // Δέσμευση μνήμης στο Device
      long long *d_row_ptr;
      int *d_col_ind;
      int *d_labels;
      bool *d_changed;
      checkCuda(cudaMalloc((void**)&d_row_ptr, sizeof(long long)*(node_number + 1)));
      checkCuda(cudaMalloc((void**)&d_col_ind, sizeof(int)*nnz));
      checkCuda(cudaMalloc((void**)&d_labels, sizeof(int)*node_number));\
      checkCuda(cudaMalloc((void**)&d_changed, sizeof(bool)));

      // Αντιγραφή δεδομένων: CPU -> GPU
      checkCuda(cudaMemcpy(d_row_ptr, h_graph->row_ptr, sizeof(long long)*(node_number + 1), cudaMemcpyHostToDevice));
      checkCuda(cudaMemcpy(d_col_ind, h_graph->col_ind, sizeof(int)*(nnz), cudaMemcpyHostToDevice));
      checkCuda(cudaMemcpy(d_labels, h_labels, sizeof(int)*(node_number), cudaMemcpyHostToDevice));

      // Ρύθμιση Threads και Blocks
      int threadsPerBlock;
      int blocksPerGrid;
      int warpsPerBlock;
      size_t shared_mem_size;
      if(strategy == 0){
        threadsPerBlock = 128;
        blocksPerGrid = (node_number + threadsPerBlock - 1) / threadsPerBlock;
        printf("Launching thread per row Kernel with: Threads/Block: %d, Blocks: %d\n", threadsPerBlock, blocksPerGrid);
      }else if(strategy == 1) {
        threadsPerBlock = 128;
        warpsPerBlock = threadsPerBlock / WARP_SIZE;
        blocksPerGrid = (node_number + warpsPerBlock - 1) / warpsPerBlock;
        printf("Launching warp per row Kernel with: Threads/Block: %d, Warps/Block: %d, Blocks: %d\n", threadsPerBlock, warpsPerBlock, blocksPerGrid);
      }else if(strategy == 2) {
        threadsPerBlock = 128;
        warpsPerBlock = threadsPerBlock / WARP_SIZE;
        blocksPerGrid = num_SMs * 8; // Αριθμος blocks ωστε Η GPU να εχει παντα δουλεια
        shared_mem_size = warpsPerBlock * sizeof(int);
        printf("Launching block per row Kernel with: Threads/Block: %d, Warps/Block: %d, Blocks: %d\n", threadsPerBlock, warpsPerBlock, blocksPerGrid);
      }


      int iterations = 0;

      checkCuda(cudaEventRecord(timer_start[strategy]));
      while(true){
        iterations++;
        // Πριν τρέξει ο kernel θέτουμε το flag σε false
        h_changed = false;
        checkCuda(cudaMemcpy(d_changed, &h_changed, sizeof(bool), cudaMemcpyHostToDevice));

        // ΕΚΤΕΛΕΣΗ ΤΟΥ KERNEL
        if(strategy == 0) thread_per_row_Kernel<<<blocksPerGrid, threadsPerBlock>>>(d_labels, d_row_ptr, d_col_ind, node_number, d_changed);
        if(strategy == 1) warp_per_row_kernel<<<blocksPerGrid, threadsPerBlock>>>(d_labels, d_row_ptr, d_col_ind, node_number, d_changed);
        if(strategy == 2) block_per_row_kernel<<<blocksPerGrid, threadsPerBlock, shared_mem_size>>>(d_labels, d_row_ptr, d_col_ind, node_number, d_changed);

        // Αντιγραφή του flag για ελεγχο
        checkCuda(cudaMemcpy(&h_changed, d_changed, sizeof(bool), cudaMemcpyDeviceToHost));

        // Έλεγχος αν ο αλγόριθμος τελείωσε
        if (!h_changed) {
          printf("Converged after %d iterations.\n", iterations);
          break;
        }

    }
    // Τελικά labels
    checkCuda(cudaMemcpy(h_labels, d_labels, sizeof(int) * node_number, cudaMemcpyDeviceToHost));

    // Υπολογισμός χρόνου
    checkCuda(cudaEventRecord(timer_stop[strategy]));
    checkCuda(cudaEventSynchronize(timer_stop[strategy]));
    const char *strategies[] = {"Thread per row", "Warp per row", "Block per row"};

    float milliseconds = 0;
    checkCuda(cudaEventElapsedTime(&milliseconds, timer_start[strategy], timer_stop[strategy]));
    printf("%s time: %f s\n", strategies[strategy], milliseconds/1000);

    int unique_labels = sort_unique_int(h_labels, node_number);
    if(unique_labels<50)  {
      printf("We have %d unique numbers in labels same as the #SCC\n", unique_labels);
      print_vector(h_labels, unique_labels);
      printf("\n");
    }
    else printf("We have %d unique numbers in labels same as the #SCC\n\n", unique_labels);


    // Απελευθέρωση Μνήμης
    cudaFree(d_row_ptr); cudaFree(d_col_ind); cudaFree(d_labels); cudaFree(d_changed);
    free(h_labels);

  }

    for (int i = 0; i < 3; i++) {
      cudaEventDestroy(timer_start[i]);
      cudaEventDestroy(timer_stop[i]);
    }
    free(timer_start);
    free(timer_stop);
    free(h_graph);
    return 0;
}

Writing GPU_cc.cu


τρέχουμε τα test που θέλουμε αλλάζοντας μόνο το όνομα του γράφου στο όρισμα.

In [ ]:
# Compile
!nvcc -arch=sm_75 -allow-unsupported-compiler GPU_cc.cu -o GPU_cc

# Run
!./GPU_cc kmer_A2a
!./GPU_cc Queen_4147
!./GPU_cc rgg_n_2_24_s0

----------- kmer_A2a -----------

Loading Graph: n=170728175, nnz=360585172
We have 40 SMs in our GPU.

Launching thread per row Kernel with: Threads/Block: 128, Blocks: 1333814
Converged after 244 iterations.
Thread per row time: 3.094354 s
We have 5353 unique numbers in labels same as the #SCC

Launching warp per row Kernel with: Threads/Block: 128, Warps/Block: 4, Blocks: 42682044
Converged after 240 iterations.
Warp per row time: 32.132740 s
We have 5353 unique numbers in labels same as the #SCC

Launching block per row Kernel with: Threads/Block: 128, Warps/Block: 4, Blocks: 320
Converged after 242 iterations.
Block per row time: 65.591370 s
We have 5353 unique numbers in labels same as the #SCC

----------- Queen_4147 -----------

Loading Graph: n=4147110, nnz=316548962
We have 40 SMs in our GPU.

Launching thread per row Kernel with: Threads/Block: 128, Blocks: 32400
Converged after 84 iterations.
Thread per row time: 5.435566 s
We have 149447 unique numbers in labels same as th

Σβηνουμε οταν τελειωσουμε το testing τα αρχεια που ανεβασαμε στο drive

In [ ]:
import os
import shutil

# ================= ΡΥΘΜΙΣΕΙΣ =================

# 1. ΑΣΦΑΛΕΙΑ: Αν είναι True, απλά τυπώνει τι θα σβήσει (δεν σβήνει τίποτα).
# Άλλαξέ το σε False όταν είσαι έτοιμος για τη διαγραφή.
DRY_RUN = False

# 2. Η βασική διαδρομή
BASE_PATH = "/content/drive/MyDrive/"

# 3. Λίστα με ΦΑΚΕΛΟΥΣ για διαγραφή (ονόματα φακέλων)
# ΠΡΟΣΟΧΗ: Θα διαγραφούν μαζί με όλα τα περιεχόμενά τους!
FOLDERS_TO_DELETE = [
    "kmer_A2a",
    "Queen_4147",
    "rgg_n_2_24_s0"
]

# 4. Λίστα με ΑΡΧΕΙΑ για διαγραφή (ονόματα αρχείων με την κατάληξη)
FILES_TO_DELETE = [
    "kmer_A2a.mat",
    "Queen_4147.mat",
    "rgg_n_2_24_s0.mat"
]

# ================= ΚΩΔΙΚΑΣ ΔΙΑΓΡΑΦΗΣ =================

def delete_items():
    print(f"--- Ξεκινάει η διαδικασία ({'SIMULATION' if DRY_RUN else 'LIVE DELETION'}) ---\n")

    # --- Βήμα 1: Διαγραφή Αρχείων ---
    print("👉 Έλεγχος Αρχείων:")
    for filename in FILES_TO_DELETE:
        full_path = os.path.join(BASE_PATH, filename)

        if os.path.exists(full_path):
            if os.path.isfile(full_path):
                if DRY_RUN:
                    print(f"   [DRY RUN] Θα διαγραφόταν το αρχείο: {full_path}")
                else:
                    try:
                        os.remove(full_path)
                        print(f"   ✅ Διαγράφηκε: {full_path}")
                    except Exception as e:
                        print(f"   ❌ Σφάλμα διαγραφής {full_path}: {e}")
            else:
                print(f"   ⚠️  Το {filename} βρέθηκε αλλά δεν είναι αρχείο (είναι φάκελος;).")
        else:
            print(f"   ⚠️  Δεν βρέθηκε: {filename}")

    print("-" * 30)

    # --- Βήμα 2: Διαγραφή Φακέλων ---
    print("👉 Έλεγχος Φακέλων:")
    for foldername in FOLDERS_TO_DELETE:
        full_path = os.path.join(BASE_PATH, foldername)

        if os.path.exists(full_path):
            if os.path.isdir(full_path):
                if DRY_RUN:
                    print(f"   [DRY RUN] Θα διαγραφόταν ο φάκελος: {full_path}")
                else:
                    try:
                        shutil.rmtree(full_path)
                        print(f"   ✅ Διαγράφηκε ο φάκελος: {full_path}")
                    except Exception as e:
                        print(f"   ❌ Σφάλμα διαγραφής {full_path}: {e}")
            else:
                print(f"   ⚠️  Το {foldername} βρέθηκε αλλά δεν είναι φάκελος.")
        else:
            print(f"   ⚠️  Δεν βρέθηκε: {foldername}")

    print("\n--- Τέλος διαδικασίας ---")

if __name__ == "__main__":
    # Έλεγχος αν έχει γίνει mount το Drive
    if not os.path.exists(BASE_PATH):
        print("❌ Το Google Drive δεν φαίνεται να είναι συνδεδεμένο.")
        print("   Τρέξε πρώτα: from google.colab import drive; drive.mount('/content/drive')")
    else:
        delete_items()

--- Ξεκινάει η διαδικασία (LIVE DELETION) ---

👉 Έλεγχος Αρχείων:
   ✅ Διαγράφηκε: /content/drive/MyDrive/kmer_A2a.mat
   ✅ Διαγράφηκε: /content/drive/MyDrive/Queen_4147.mat
   ✅ Διαγράφηκε: /content/drive/MyDrive/rgg_n_2_24_s0.mat
------------------------------
👉 Έλεγχος Φακέλων:
   ✅ Διαγράφηκε ο φάκελος: /content/drive/MyDrive/kmer_A2a
   ✅ Διαγράφηκε ο φάκελος: /content/drive/MyDrive/Queen_4147
   ✅ Διαγράφηκε ο φάκελος: /content/drive/MyDrive/rgg_n_2_24_s0

--- Τέλος διαδικασίας ---
